In [42]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy import optimize
from scipy.optimize import fsolve
import math
import pyvisa #para comunicar com os geradores de pulsos/corrente       
#import usb.core # comunicação usb
import time #operações relacionadas com tempo
import urllib.request #para abrir URLs 
import functools as ft #operações complexas
import multiprocessing #para aumentar velocidade de cálculo
import threading # para aumentar velocidade de cálculo
import queue # para aumentar velocidade de cálculo
import random
from IPython.display import display #for the display of the current k and counts

In [25]:
rm = pyvisa.ResourceManager()

instrument = rm.open_resource('USB0::0x0699::0x0358::C018403::INSTR')
keithley = rm.open_resource('USB0::0x05E6::0x2450::04608397::INSTR')

In [26]:
def function(a): ##This function removes the unwanted strings from the Yokogawa Values converting them into floats
    assert type(a) == str
    b = list(a.strip())
    return float(ft.reduce(lambda x, y: x + y, list(filter(lambda x: x.isdigit() or x == '.' or x == '-', b))))


In [27]:
def values(website, positions, result_queue): ###This function helps us get the values from the Yokogawa by putting its site in what positions are the the values taken from it and then put it in a queue for multithreading
    web_url = urllib.request.urlopen(website)
    content = str(web_url.read())
    values_list = [function(content[i[0]:i[1]]) for i in positions] #positions is the list of lists with the voltage readings
    result_queue.put(values_list) #multithreading
a=time.perf_counter()
print(time.perf_counter()-a)

4.33000095654279e-05


In [28]:
def AFG_signals(p,limit,initial_values,result_queue): 
    b=time.time()
    c=time.perf_counter()
    w=0
    while time.time()-b<limit:
         ## continua a mandar sinais de tensao at]e 1MHz para a source Range, limit é dado pelo tempo médio de leitura do yokogawa
        initial_values=coefficients_solver(initial_values,abs(time.perf_counter()-c+w),p) #coefficients_solver resolver o sistema de ODE's
        c=(time.perf_counter())
        w=initial_values[-1]
        del initial_values[-1]
        current = 100*10**-12/(2*10**4)*initial_values[0] #current-counts proportionality, 20kcps<->100pA
    if initial_values[0]<2*10**4: #limite do loglin é 20kcps
        instrument.write('SOUR1:FREQ '+str(initial_values[0])+'Hz') #writes in the afg the value of the counts given by the coefficients_solver
        instrument.write('SOUR2:FREQ '+str(initial_values[0])+'Hz')
        instrument.write('SOUR1:PULS:WIDTH 200ns') #para simular fission chamber
        instrument.write('SOUR2:PULS:WIDTH 200ns')
    elif 2*10**4<initial_values[0]<9*10**5: #limite do source range é 10^6
        instrument.write('SOUR1:FREQ '+str(initial_values[0])+'Hz') #writes in the afg the value of the counts given by the coefficients_solver
        instrument.write('SOUR2:FREQ '+str(initial_values[0])+'Hz')
        instrument.write('SOUR1:PULS:WIDTH 200ns') #para simular fission chamber
        instrument.write('SOUR2:PULS:WIDTH 200ns')
        keithley.write(f':SOUR:CURR {current}')  #set the source current to desired value
        #keithley.write('DISPlay:SCReen SWIPE_USER') #activate display text in the instrument
        #keithley.write(f'DISPlay:USER1:TEXT "CURR: {current:.5e} A"') #show the current being sourced in text format. this was done due to display problems

    ### Eduardo tu depois vais ter de por o comando para o gerador de sinais, usar mesma função para a fonte de corrente se possível
    else:
        instrument.write('SOUR1:FREQ '+str(1000000*(1+(0.1-0.2*random.random())))+'Hz')
        instrument.write('SOUR2:FREQ '+str(1000000*(1+(0.1-0.2*random.random())))+'Hz')
        instrument.write('SOUR1:PULS:WIDTH 200ns')
        instrument.write('SOUR2:PULS:WIDTH 200ns')
        keithley.write(f':SOUR:CURR {current}')  #set the source current to desired value
        #keithley.write('DISPlay:SCReen SWIPE_USER') #activate display text in the instrument
        #keithley.write(f'DISPlay:USER1:TEXT "CURR: {current:.5e} A"') #show the current being sourced in text format. this was done due to display problems
    result_queue.put(initial_values) #multithreading 

In [29]:
def threadings(): ### Thread Inicial quando está no Subcritico Só lê o valor do SALMAO e do CARAPAU
    if __name__ == "__main__": #não recebe os valores das ODE's porque no canal de arranque basta usar a progressão geométrica. não se resolvem as ODE's
        positions1 = [[2321, 2331], [3129, 3139], [3937, 3947], [4745, 4755], [5553, 5563], [6361, 6371]] #positions where the values of the readings may be
        positions2 = [[2339, 2346], [3145, 3152], [3951, 3959], [4758, 4766]]

        result_queue = queue.Queue()

        t1 = threading.Thread(target=values, args=('http://10.10.15.20/cgi-bin/moni/allch.cgi', positions1, result_queue))
        t2 = threading.Thread(target=values, args=('http://10.10.15.23/cgi-bin/ope/allch.cgi', positions2, result_queue))
    
        t1.start() #start the threading
        t2.start()
        

        t1.join() #joins the threading, output is given when both readings are complete
        t2.join()
        

        results = []
        while not result_queue.empty():
            results.append(result_queue.get())
    if len(results[1])>len(results[0]):
        results=[results[1],results[0]] #ensure that the order of the readings is [carapau, salmao]
    return results #readings from carapau and salmao

In [30]:
def threadings1(p, limit, initial_values): #Quando passa para o estado supercritico Posição das barras mede e depois lê no Yokogawa ao mesmo tempo que está a ler ele corre as equações para temos um tempo mais pequeno
    if __name__ == "__main__":           #função igual à anterior mas recebe os valores das ODE's porque já é preciso calcular
        positions1 = [[2321, 2331], [3129, 3139], [3937, 3947], [4745, 4755], [5553, 5563], [6361, 6371]]
        positions2 = [[2339, 2346], [3145, 3152], [3951, 3959], [4758, 4766]]

        result_queue = queue.Queue()
        result_queue1= queue.Queue()

        t1 = threading.Thread(target=values, args=('http://10.10.15.20/cgi-bin/moni/allch.cgi', positions1, result_queue))
        t2 = threading.Thread(target=values, args=('http://10.10.15.23/cgi-bin/ope/allch.cgi', positions2, result_queue)) # Por tudo em um Thread.
        t3 = threading.Thread(target=AFG_signals, args=(p,limit,initial_values,result_queue1))

        t1.start()
        t2.start()
        t3.start()
        
        t1.join()
        t2.join()
        t3.join()
        
        # Collect results from the queue
        results = []
        while not result_queue.empty():
            results.append(result_queue.get())
    if len(results[1])>len(results[0]):
        results=[results[1],results[0]]
    results.append(result_queue1.get())
    return results

In [31]:
#### Valores experimentais do RPI
Lambda = [0.0127, 0.0317, 0.116, 0.311, 1.4, 3.87] ## constante de decaimento
beta = [0.00031, 0.00166, 0.00151, 0.00328, 0.00103, 0.00021]
l = 0.00002 ### prompt neutrons
epsilon=1

ld=l*(1-sum(beta))+beta[0]*(1/Lambda[0])+beta[1]*(1/Lambda[1])+beta[2]*(1/Lambda[2])+beta[3]*(1/Lambda[3])+beta[4]*(1/Lambda[4])+beta[5]*(1/Lambda[5]) #slow decay lifetime
#######

In [32]:
#### As sete funções kinematicas do comportamento dos neutrões 
N=lambda  n,c1,c2,c3,c4,c5,c6,p: ((p-sum(beta))/l)*n+0.0127*c1+0.0317*c2+0.116*c3+0.311*c4+1.40*c5+3.87*c6
C1=lambda n,c1,c2,c3,c4,c5,c6: (beta[0]/l)*n-0.0127*c1-0.0*c2-0*c3-0*c4-0*c5-0*c6
C2=lambda n,c1,c2,c3,c4,c5,c6: (beta[1]/l)*n-0*c1-0.0317*c2-0*c3-0*c4-0*c5-0*c6
C3=lambda n,c1,c2,c3,c4,c5,c6: (beta[2]/l)*n-0*c1-0*c2-0.116*c3-0*c4-0*c5-0*c6
C4=lambda n,c1,c2,c3,c4,c5,c6: (beta[3]/l)*n-0.0*c1-0.0*c2-0*c3-0.311*c4-0*c5-0*c6
C5=lambda n,c1,c2,c3,c4,c5,c6: (beta[4]/l)*n-0.0*c1-0.0*c2-0.0*c3-0*c4-1.40*c5-0*c6
C6=lambda n,c1,c2,c3,c4,c5,c6: (beta[5]/l)*n-0.0*c1-0.0*c2-0.0*c3-0.0*c4-0*c5-3.87*c6

In [33]:
def root_mean_squared(x):
    assert type(x)==list
    return (sum(list(map(lambda y: y**2,x)))/len(x))**.5

In [34]:
### Cubic spline  with the values of the teachers documents
#to know the p corresponding to the rods position
x=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8.0, 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9.0, 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 10.0, 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11.0, 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12.0, 12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13.0, 13.1, 13.2, 13.3, 13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 14.0, 14.1, 14.2, 14.3, 14.4, 14.5, 14.6, 14.7, 14.8, 14.9, 15.0, 15.1, 15.2, 15.3, 15.4, 15.5, 15.6, 15.7, 15.8, 15.9, 16.0, 16.1, 16.2, 16.3, 16.4, 16.5, 16.6, 16.7, 16.8, 16.9, 17.0, 17.1, 17.2, 17.3, 17.4, 17.5, 17.6, 17.7, 17.8, 17.9, 18.0, 18.1, 18.2, 18.3, 18.4, 18.5, 18.6, 18.7, 18.8, 18.9, 19.0, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20.0, 20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9, 21.0, 21.1, 21.2, 21.3, 21.4, 21.5, 21.6, 21.7, 21.8, 21.9, 22.0, 22.1, 22.2, 22.3, 22.4, 22.5, 22.6, 22.7, 22.8, 22.9, 23.0, 23.1, 23.2, 23.3, 23.4, 23.5, 23.6, 23.7, 23.8, 23.9, 24.0, 24.1, 24.2, 24.3, 24.4, 24.5, 24.6, 24.7, 24.8, 24.9, 25.0, 25.1, 25.2, 25.3, 25.4, 25.5, 25.6, 25.7, 25.8, 25.9, 26.0, 26.1, 26.2, 26.3, 26.4, 26.5, 26.6, 26.7, 26.8, 26.9, 27.0, 27.1, 27.2, 27.3, 27.4, 27.5, 27.6, 27.7, 27.8, 27.9, 28.0, 28.1, 28.2, 28.3, 28.4, 28.5, 28.6, 28.7, 28.8, 28.9, 29.0, 29.1, 29.2, 29.3, 29.4, 29.5, 29.6, 29.7, 29.8, 29.9, 30.0, 30.1, 30.2, 30.3, 30.4, 30.5, 30.6, 30.7, 30.8, 30.9, 31.0, 31.1, 31.2, 31.3, 31.4, 31.5, 31.6, 31.7, 31.8, 31.9, 32.0, 32.1, 32.2, 32.3, 32.4, 32.5, 32.6, 32.7, 32.8, 32.9, 33.0, 33.1, 33.2, 33.3, 33.4, 33.5, 33.6, 33.7, 33.8, 33.9, 34.0, 34.1, 34.2, 34.3, 34.4, 34.5, 34.6, 34.7, 34.8, 34.9, 35.0, 35.1, 35.2, 35.3, 35.4, 35.5, 35.6, 35.7, 35.8, 35.9, 36.0, 36.1, 36.2, 36.3, 36.4, 36.5, 36.6, 36.7, 36.8, 36.9, 37.0, 37.1, 37.2, 37.3, 37.4, 37.5, 37.6, 37.7, 37.8, 37.9, 38.0, 38.1, 38.2, 38.3, 38.4, 38.5, 38.6, 38.7, 38.8, 38.9, 39.0, 39.1, 39.2, 39.3, 39.4, 39.5, 39.6, 39.7, 39.8, 39.9, 40.0, 40.1, 40.2, 40.3, 40.4, 40.5, 40.6, 40.7, 40.8, 40.9, 41.0, 41.1, 41.2, 41.3, 41.4, 41.5, 41.6, 41.7, 41.8, 41.9, 42.0, 42.1, 42.2, 42.3, 42.4, 42.5, 42.6, 42.7, 42.8, 42.9, 43.0, 43.1, 43.2, 43.3, 43.4, 43.5, 43.6, 43.7, 43.8, 43.9, 44.0, 44.1, 44.2, 44.3, 44.4, 44.5, 44.6, 44.7, 44.8, 44.9, 45.0, 45.1, 45.2, 45.3, 45.4, 45.5, 45.6, 45.7, 45.8, 45.9, 46.0, 46.1, 46.2, 46.3, 46.4, 46.5, 46.6, 46.7, 46.8, 46.9, 47.0, 47.1, 47.2, 47.3, 47.4, 47.5, 47.6, 47.7, 47.8, 47.9, 48.0, 48.1, 48.2, 48.3, 48.4, 48.5, 48.6, 48.7, 48.8, 48.9, 49.0, 49.1, 49.2, 49.3, 49.4, 49.5, 49.6, 49.7, 49.8, 49.9, 50.0, 50.1, 50.2, 50.3, 50.4, 50.5, 50.6, 50.7, 50.8, 50.9, 51.0, 51.1, 51.2, 51.3, 51.4, 51.5, 51.6, 51.7, 51.8, 51.9, 52.0, 52.1, 52.2, 52.3, 52.4, 52.5, 52.6, 52.7, 52.8, 52.9, 53.0, 53.1, 53.2, 53.3, 53.4, 53.5, 53.6, 53.7, 53.8, 53.9, 54.0, 54.1, 54.2, 54.3, 54.4, 54.5, 54.6, 54.7, 54.8, 54.9, 55.0, 55.1, 55.2, 55.3, 55.4, 55.5, 55.6, 55.7, 55.8, 55.9, 56.0, 56.1, 56.2, 56.3, 56.4, 56.5, 56.6, 56.7, 56.8, 56.9, 57.0, 57.1, 57.2, 57.3, 57.4, 57.5, 57.6, 57.7, 57.8, 57.9, 58.0, 58.1, 58.2, 58.3, 58.4, 58.5, 58.6, 58.7, 58.8, 58.9, 59.0, 59.1, 59.2, 59.3, 59.4, 59.5, 59.6, 59.7, 59.8, 59.9, 60.0, 60.1, 60.2, 60.3, 60.4, 60.5, 60.6, 60.7, 60.8, 60.9, 61.0, 61.1, 61.2, 61.3, 61.4, 61.5, 61.6, 61.7, 61.8, 61.9, 62.0, 62.1, 62.2, 62.3, 62.4, 62.5, 62.6, 62.7, 62.8, 62.9, 63.0, 63.1, 63.2, 63.3, 63.4, 63.5, 63.6, 63.7, 63.8, 63.9, 64.0, 64.1, 64.2, 64.3, 64.4, 64.5, 64.6, 64.7, 64.8, 64.9, 65.0, 65.1, 65.2, 65.3, 65.4, 65.5, 65.6, 65.7, 65.8, 65.9, 66.0, 66.1, 66.2, 66.3, 66.4, 66.5, 66.6, 66.7, 66.8, 66.9, 67.0, 67.1, 67.2, 67.3, 67.4, 67.5, 67.6, 67.7, 67.8, 67.9, 68.0, 68.1, 68.2, 68.3, 68.4, 68.5, 68.6, 68.7, 68.8, 68.9, 69.0, 69.1, 69.2, 69.3, 69.4, 69.5, 69.6, 69.7, 69.8, 69.9, 70.0, 70.1, 70.2, 70.3, 70.4, 70.5, 70.6, 70.7, 70.8, 70.9, 71.0, 71.1, 71.2, 71.3, 71.4, 71.5, 71.6, 71.7, 71.8, 71.9, 72.0, 72.1, 72.2, 72.3, 72.4, 72.5, 72.6, 72.7, 72.8, 72.9, 73.0, 73.1, 73.2, 73.3, 73.4, 73.5, 73.6, 73.7, 73.8, 73.9, 74.0, 74.1, 74.2, 74.3, 74.4, 74.5, 74.6, 74.7, 74.8, 74.9, 75.0, 75.1, 75.2, 75.3, 75.4, 75.5, 75.6, 75.7, 75.8, 75.9, 76.0, 76.1, 76.2, 76.3, 76.4, 76.5, 76.6, 76.7, 76.8, 76.9, 77.0, 77.1, 77.2, 77.3, 77.4, 77.5, 77.6, 77.7, 77.8, 77.9, 78.0, 78.1, 78.2, 78.3, 78.4, 78.5, 78.6, 78.7, 78.8, 78.9, 79.0, 79.1, 79.2, 79.3, 79.4, 79.5, 79.6, 79.7, 79.8, 79.9, 80.0, 80.1, 80.2, 80.3, 80.4, 80.5, 80.6, 80.7, 80.8, 80.9, 81.0, 81.1, 81.2, 81.3, 81.4, 81.5, 81.6, 81.7, 81.8, 81.9, 82.0, 82.1, 82.2, 82.3, 82.4, 82.5, 82.6, 82.7, 82.8, 82.9, 83.0, 83.1, 83.2, 83.3, 83.4, 83.5, 83.6, 83.7, 83.8, 83.9, 84.0, 84.1, 84.2, 84.3, 84.4, 84.5, 84.6, 84.7, 84.8, 84.9, 85.0, 85.1, 85.2, 85.3, 85.4, 85.5, 85.6, 85.7, 85.8, 85.9, 86.0, 86.1, 86.2, 86.3, 86.4, 86.5, 86.6, 86.7, 86.8, 86.9, 87.0, 87.1, 87.2, 87.3, 87.4, 87.5, 87.6, 87.7, 87.8, 87.9, 88.0, 88.1, 88.2, 88.3, 88.4, 88.5, 88.6, 88.7, 88.8, 88.9, 89.0, 89.1, 89.2, 89.3, 89.4, 89.5, 89.6, 89.7, 89.8, 89.9, 90.0, 90.1, 90.2, 90.3, 90.4, 90.5, 90.6, 90.7, 90.8, 90.9, 91.0, 91.1, 91.2, 91.3, 91.4, 91.5, 91.6, 91.7, 91.8, 91.9, 92.0, 92.1, 92.2, 92.3, 92.4, 92.5, 92.6, 92.7, 92.8, 92.9, 93.0, 93.1, 93.2, 93.3, 93.4, 93.5, 93.6, 93.7, 93.8, 93.9, 94.0, 94.1, 94.2, 94.3, 94.4, 94.5, 94.6, 94.7, 94.8, 94.9, 95.0, 95.1, 95.2, 95.3, 95.4, 95.5, 95.6, 95.7, 95.8, 95.9, 96.0, 96.1, 96.2, 96.3, 96.4, 96.5, 96.6, 96.7, 96.8, 96.9, 97.0, 97.1, 97.2, 97.3, 97.4, 97.5, 97.6, 97.7, 97.8, 97.9, 98.0, 98.1, 98.2, 98.3, 98.4, 98.5, 98.6, 98.7, 98.8, 98.9, 99.0, 99.1, 99.2, 99.3, 99.4, 99.5, 99.6, 99.7, 99.8, 99.9]
y1=[0.0, 0.4, 0.8, 1.2, 1.7, 2.1, 2.5, 2.9, 3.3, 3.8, 4.2, 4.6, 5.0, 5.5, 5.9, 6.3, 6.8, 7.2, 7.7, 8.1, 8.6, 9.0, 9.5, 9.9, 10.4, 10.9, 11.3, 11.8, 12.3, 12.8, 13.3, 13.7, 14.2, 14.7, 15.2, 15.8, 16.3, 16.8, 17.3, 17.8, 18.4, 18.9, 19.5, 20.0, 20.6, 21.1, 21.7, 22.3, 22.8, 23.4, 24.0, 24.6, 25.2, 25.8, 26.4, 27.1, 27.7, 28.3, 29.0, 29.6, 30.3, 31.0, 31.6, 32.3, 33.0, 33.7, 34.4, 35.1, 35.8, 36.6, 37.3, 38.1, 38.8, 39.6, 40.4, 41.1, 41.9, 42.7, 43.5, 44.4, 45.2, 46.0, 46.9, 47.7, 48.6, 49.5, 50.4, 51.3, 52.2, 53.1, 54.0, 55.0, 55.9, 56.9, 57.9, 58.8, 59.8, 60.8, 61.9, 62.9, 63.9, 65.0, 66.0, 67.1, 68.2, 69.3, 70.4, 71.6, 72.7, 73.8, 75.0, 76.2, 77.4, 78.6, 79.8, 81.0, 82.3, 83.5, 84.8, 86.1, 87.4, 88.7, 90.0, 91.3, 92.7, 94.1, 95.4, 96.8, 98.3, 99.7, 101.1, 102.6, 104.1, 105.5, 107.0, 108.6, 110.1, 111.6, 113.2, 114.8, 116.4, 118.0, 119.6, 121.3, 122.9, 124.6, 126.3, 128.0, 129.7, 131.5, 133.2, 135.0, 136.8, 138.6, 140.5, 142.3, 144.2, 146.1, 148.0, 149.9, 151.8, 153.8, 155.8, 157.8, 159.8, 161.8, 163.8, 165.9, 168.0, 170.1, 172.2, 174.4, 176.5, 178.7, 180.9, 183.2, 185.4, 187.7, 189.9, 192.3, 194.6, 196.9, 199.3, 201.7, 204.1, 206.5, 209.0, 211.4, 213.9, 216.4, 219.0, 221.5, 224.1, 226.7, 229.3, 231.9, 234.6, 237.3, 240.0, 242.7, 245.5, 248.3, 251.1, 253.9, 256.7, 259.6, 262.5, 265.4, 268.3, 271.3, 274.3, 277.3, 280.3, 283.4, 286.4, 289.5, 292.7, 295.8, 299.0, 302.2, 305.4, 308.7, 311.9, 315.2, 318.6, 321.9, 325.3, 328.7, 332.1, 335.5, 338.9, 342.4, 345.9, 349.4, 353.0, 356.5, 360.1, 363.7, 367.3, 371.0, 374.6, 378.3, 382.0, 385.7, 389.4, 393.2, 397.0, 400.7, 404.5, 408.4, 412.2, 416.0, 419.9, 423.8, 427.7, 431.6, 435.5, 439.5, 443.4, 447.4, 451.4, 455.4, 459.4, 463.5, 467.5, 471.6, 475.6, 479.7, 483.8, 487.9, 492.0, 496.2, 500.3, 504.4, 508.6, 512.8, 517.0, 521.1, 525.3, 529.6, 533.8, 538.0, 542.2, 546.5, 550.7, 555.0, 559.3, 563.6, 567.8, 572.1, 576.4, 580.7, 585.0, 589.4, 593.7, 598.0, 602.3, 606.7, 611.0, 615.4, 619.7, 624.1, 628.4, 632.8, 637.2, 641.6, 645.9, 650.3, 654.7, 659.1, 663.5, 668.0, 672.4, 676.8, 681.2, 685.7, 690.1, 694.6, 699.0, 703.5, 708.0, 712.4, 716.9, 721.4, 725.9, 730.4, 734.9, 739.4, 743.9, 748.5, 753.0, 757.5, 762.1, 766.6, 771.2, 775.8, 780.3, 784.9, 789.5, 794.1, 798.7, 803.3, 807.9, 812.5, 817.2, 821.8, 826.4, 831.1, 835.7, 840.4, 845.1, 849.8, 854.5, 859.2, 863.9, 868.6, 873.3, 878.0, 882.7, 887.5, 892.2, 897.0, 901.8, 906.5, 911.3, 916.1, 920.9, 925.7, 930.5, 935.3, 940.2, 945.0, 949.8, 954.7, 959.5, 964.4, 969.2, 974.1, 979.0, 983.9, 988.7, 993.6, 998.5, 1003.4, 1008.3, 1013.2, 1018.1, 1023.0, 1027.9, 1032.9, 1037.8, 1042.7, 1047.6, 1052.6, 1057.5, 1062.4, 1067.4, 1072.3, 1077.3, 1082.2, 1087.1, 1092.1, 1097.0, 1102.0, 1106.9, 1111.9, 1116.8, 1121.8, 1126.7, 1131.7, 1136.6, 1141.6, 1146.5, 1151.5, 1156.4, 1161.4, 1166.3, 1171.3, 1176.2, 1181.2, 1186.1, 1191.1, 1196.0, 1200.9, 1205.9, 1210.8, 1215.7, 1220.6, 1225.6, 1230.5, 1235.4, 1240.3, 1245.2, 1250.1, 1255.0, 1259.9, 1264.8, 1269.7, 1274.6, 1279.4, 1284.3, 1289.2, 1294.0, 1298.9, 1303.7, 1308.6, 1313.4, 1318.2, 1323.1, 1327.9, 1332.7, 1337.5, 1342.3, 1347.0, 1351.8, 1356.6, 1361.3, 1366.1, 1370.8, 1375.6, 1380.3, 1385.0, 1389.7, 1394.4, 1399.1, 1403.8, 1408.5, 1413.1, 1417.8, 1422.4, 1427.0, 1431.6, 1436.2, 1440.8, 1445.4, 1450.0, 1454.5, 1459.1, 1463.6, 1468.1, 1472.6, 1477.1, 1481.6, 1486.1, 1490.5, 1495.0, 1499.4, 1503.8, 1508.2, 1512.6, 1517.0, 1521.4, 1525.7, 1530.1, 1534.4, 1538.7, 1543.0, 1547.3, 1551.6, 1555.8, 1560.1, 1564.3, 1568.5, 1572.8, 1577.0, 1581.2, 1585.3, 1589.5, 1593.7, 1597.8, 1602.0, 1606.1, 1610.2, 1614.3, 1618.4, 1622.5, 1626.6, 1630.6, 1634.7, 1638.7, 1642.8, 1646.8, 1650.8, 1654.8, 1658.8, 1662.8, 1666.8, 1670.8, 1674.7, 1678.7, 1682.6, 1686.6, 1690.5, 1694.4, 1698.4, 1702.3, 1706.2, 1710.0, 1713.9, 1717.8, 1721.7, 1725.5, 1729.4, 1733.2, 1737.1, 1740.9, 1744.8, 1748.6, 1752.4, 1756.2, 1760.0, 1763.8, 1767.6, 1771.4, 1775.2, 1778.9, 1782.7, 1786.5, 1790.2, 1794.0, 1797.7, 1801.5, 1805.2, 1809.0, 1812.7, 1816.4, 1820.1, 1823.8, 1827.6, 1831.3, 1835.0, 1838.6, 1842.3, 1846.0, 1849.7, 1853.3, 1857.0, 1860.7, 1864.3, 1867.9, 1871.6, 1875.2, 1878.8, 1882.4, 1886.1, 1889.7, 1893.3, 1896.8, 1900.4, 1904.0, 1907.6, 1911.1, 1914.7, 1918.2, 1921.7, 1925.3, 1928.8, 1932.3, 1935.8, 1939.3, 1942.8, 1946.3, 1949.7, 1953.2, 1956.7, 1960.1, 1963.5, 1967.0, 1970.4, 1973.8, 1977.2, 1980.6, 1984.0, 1987.3, 1990.7, 1994.1, 1997.4, 2000.7, 2004.1, 2007.4, 2010.7, 2014.0, 2017.3, 2020.6, 2023.8, 2027.1, 2030.3, 2033.6, 2036.8, 2040.0, 2043.2, 2046.4, 2049.6, 2052.8, 2055.9, 2059.1, 2062.2, 2065.3, 2068.5, 2071.6, 2074.7, 2077.7, 2080.8, 2083.9, 2086.9, 2090.0, 2093.0, 2096.0, 2099.0, 2102.0, 2105.0, 2107.9, 2110.9, 2113.8, 2116.8, 2119.7, 2122.6, 2125.5, 2128.4, 2131.2, 2134.1, 2136.9, 2139.8, 2142.6, 2145.4, 2148.2, 2151.0, 2153.8, 2156.5, 2159.3, 2162.0, 2164.8, 2167.5, 2170.2, 2172.9, 2175.6, 2178.3, 2180.9, 2183.6, 2186.2, 2188.9, 2191.5, 2194.1, 2196.7, 2199.3, 2201.8, 2204.4, 2207.0, 2209.5, 2212.0, 2214.6, 2217.1, 2219.6, 2222.1, 2224.5, 2227.0, 2229.5, 2231.9, 2234.3, 2236.8, 2239.2, 2241.6, 2244.0, 2246.4, 2248.7, 2251.1, 2253.4, 2255.8, 2258.1, 2260.4, 2262.8, 2265.0, 2267.3, 2269.6, 2271.9, 2274.1, 2276.4, 2278.6, 2280.9, 2283.1, 2285.3, 2287.5, 2289.7, 2291.9, 2294.0, 2296.2, 2298.3, 2300.5, 2302.6, 2304.7, 2306.8, 2308.9, 2311.0, 2313.1, 2315.2, 2317.2, 2319.3, 2321.3, 2323.3, 2325.4, 2327.4, 2329.4, 2331.4, 2333.4, 2335.3, 2337.3, 2339.2, 2341.2, 2343.1, 2345.1, 2347.0, 2348.9, 2350.8, 2352.7, 2354.6, 2356.4, 2358.3, 2360.1, 2362.0, 2363.8, 2365.7, 2367.5, 2369.3, 2371.1, 2372.9, 2374.7, 2376.4, 2378.2, 2379.9, 2381.7, 2383.4, 2385.2, 2386.9, 2388.6, 2390.3, 2392.0, 2393.7, 2395.4, 2397.0, 2398.7, 2400.3, 2402.0, 2403.6, 2405.2, 2406.9, 2408.5, 2410.1, 2411.7, 2413.2, 2414.8, 2416.4, 2417.9, 2419.5, 2421.0, 2422.6, 2424.1, 2425.6, 2427.1, 2428.6, 2430.1, 2431.6, 2433.1, 2434.5, 2436.0, 2437.4, 2438.9, 2440.3, 2441.7, 2443.1, 2444.6, 2446.0, 2447.3, 2448.7, 2450.1, 2451.5, 2452.8, 2454.2, 2455.5, 2456.9, 2458.2, 2459.5, 2460.8, 2462.1, 2463.4, 2464.7, 2466.0, 2467.2, 2468.5, 2469.7, 2471.0, 2472.2, 2473.4, 2474.7, 2475.9, 2477.1, 2478.3, 2479.5, 2480.6, 2481.8, 2483.0, 2484.1, 2485.3, 2486.4, 2487.6, 2488.7, 2489.8, 2490.9, 2492.0, 2493.1, 2494.2, 2495.3, 2496.3, 2497.4, 2498.4, 2499.5, 2500.5, 2501.6, 2502.6, 2503.6, 2504.6, 2505.6, 2506.6, 2507.6, 2508.5, 2509.5, 2510.5, 2511.4, 2512.4, 2513.3, 2514.2, 2515.2, 2516.1, 2517.0, 2517.9, 2518.8, 2519.6, 2520.5, 2521.4, 2522.2, 2523.1, 2523.9, 2524.8, 2525.6, 2526.4, 2527.2, 2528.1, 2528.8, 2529.6, 2530.4, 2531.2, 2532.0, 2532.7, 2533.5, 2534.2, 2535.0, 2535.7, 2536.4, 2537.1, 2537.9, 2538.6, 2539.3, 2539.9, 2540.6, 2541.3, 2542.0, 2542.6, 2543.3, 2543.9, 2544.5, 2545.2, 2545.8, 2546.4, 2547.0, 2547.6, 2548.2, 2548.8, 2549.4, 2549.9, 2550.5, 2551.1, 2551.6, 2552.1, 2552.7, 2553.2, 2553.7, 2554.2, 2554.7, 2555.2, 2555.7, 2556.2, 2556.7, 2557.2, 2557.6, 2558.1, 2558.5, 2559.0, 2559.4, 2559.8, 2560.3, 2560.7, 2561.1, 2561.5, 2561.9, 2562.3, 2562.6, 2563.0, 2563.4, 2563.7, 2564.1, 2564.4, 2564.8, 2565.1, 2565.4, 2565.7, 2566.0, 2566.3, 2566.6, 2566.9, 2567.2, 2567.5, 2567.8, 2568.0, 2568.3, 2568.5, 2568.8, 2569.0, 2569.2, 2569.4, 2569.7, 2569.9, 2570.1, 2570.3, 2570.4, 2570.6, 2570.8, 2571.0, 2571.1, 2571.3, 2571.4, 2571.6, 2571.7, 2571.8, 2571.9, 2572.1, 2572.2, 2572.3, 2572.4, 2572.5, 2572.5, 2572.6, 2572.7, 2572.7, 2572.8, 2572.8, 2572.9, 2572.9, 2572.9, 2573.0, 2573.0, 2573.0]
y2=[0.0, 0.4, 0.8, 1.2, 1.6, 2.0, 2.5, 2.9, 3.3, 3.7, 4.2, 4.6, 5.0, 5.5, 5.9, 6.4, 6.8, 7.3, 7.8, 8.2, 8.7, 9.2, 9.7, 10.2, 10.7, 11.2, 11.7, 12.2, 12.7, 13.2, 13.7, 14.3, 14.8, 15.4, 15.9, 16.5, 17.0, 17.6, 18.2, 18.8, 19.4, 20.0, 20.6, 21.2, 21.8, 22.4, 23.1, 23.7, 24.4, 25.0, 25.7, 26.4, 27.0, 27.7, 28.4, 29.1, 29.9, 30.6, 31.3, 32.1, 32.8, 33.6, 34.3, 35.1, 35.9, 36.7, 37.5, 38.3, 39.2, 40.0, 40.8, 41.7, 42.6, 43.5, 44.3, 45.2, 46.2, 47.1, 48.0, 49.0, 49.9, 50.9, 51.9, 52.9, 53.9, 54.9, 55.9, 56.9, 58.0, 59.0, 60.1, 61.2, 62.3, 63.4, 64.5, 65.7, 66.8, 68.0, 69.2, 70.4, 71.6, 72.8, 74.0, 75.3, 76.5, 77.8, 79.1, 80.4, 81.7, 83.0, 84.4, 85.7, 87.1, 88.5, 89.9, 91.3, 92.7, 94.2, 95.6, 97.1, 98.6, 100.1, 101.6, 103.2, 104.7, 106.3, 107.9, 109.5, 111.1, 112.8, 114.4, 116.1, 117.8, 119.5, 121.2, 123.0, 124.7, 126.5, 128.3, 130.1, 131.9, 133.8, 135.6, 137.5, 139.4, 141.3, 143.3, 145.2, 147.2, 149.2, 151.2, 153.2, 155.3, 157.4, 159.4, 161.6, 163.7, 165.8, 168.0, 170.2, 172.4, 174.6, 176.9, 179.1, 181.4, 183.7, 186.0, 188.4, 190.8, 193.1, 195.6, 198.0, 200.4, 202.9, 205.4, 207.9, 210.5, 213.0, 215.6, 218.2, 220.8, 223.5, 226.2, 228.9, 231.6, 234.3, 237.1, 239.9, 242.7, 245.5, 248.4, 251.2, 254.1, 257.1, 260.0, 263.0, 266.0, 269.0, 272.0, 275.1, 278.2, 281.3, 284.4, 287.6, 290.8, 294.0, 297.2, 300.5, 303.7, 307.0, 310.3, 313.7, 317.1, 320.4, 323.9, 327.3, 330.8, 334.2, 337.7, 341.3, 344.8, 348.4, 352.0, 355.6, 359.2, 362.9, 366.6, 370.3, 374.0, 377.7, 381.5, 385.3, 389.1, 392.9, 396.8, 400.7, 404.6, 408.5, 412.4, 416.4, 420.4, 424.4, 428.4, 432.5, 436.6, 440.6, 444.8, 448.9, 453.1, 457.2, 461.4, 465.7, 469.9, 474.2, 478.4, 482.7, 487.1, 491.4, 495.8, 500.2, 504.6, 509.0, 513.4, 517.9, 522.4, 526.9, 531.4, 536.0, 540.5, 545.1, 549.7, 554.4, 559.0, 563.7, 568.4, 573.1, 577.8, 582.5, 587.3, 592.1, 596.9, 601.7, 606.5, 611.4, 616.2, 621.1, 626.0, 630.9, 635.9, 640.8, 645.8, 650.7, 655.7, 660.7, 665.7, 670.8, 675.8, 680.8, 685.9, 691.0, 696.1, 701.2, 706.3, 711.4, 716.5, 721.7, 726.8, 732.0, 737.2, 742.4, 747.5, 752.7, 758.0, 763.2, 768.4, 773.6, 778.9, 784.1, 789.3, 794.6, 799.9, 805.1, 810.4, 815.7, 820.9, 826.2, 831.5, 836.8, 842.1, 847.4, 852.7, 858.0, 863.3, 868.6, 873.9, 879.2, 884.5, 889.9, 895.2, 900.5, 905.8, 911.1, 916.4, 921.7, 927.0, 932.4, 937.7, 943.0, 948.3, 953.6, 958.9, 964.2, 969.5, 974.8, 980.1, 985.4, 990.7, 996.0, 1001.3, 1006.5, 1011.8, 1017.1, 1022.3, 1027.6, 1032.9, 1038.1, 1043.4, 1048.6, 1053.8, 1059.1, 1064.3, 1069.5, 1074.7, 1079.9, 1085.1, 1090.3, 1095.5, 1100.7, 1105.8, 1111.0, 1116.1, 1121.3, 1126.4, 1131.5, 1136.6, 1141.7, 1146.8, 1151.9, 1157.0, 1162.1, 1167.1, 1172.2, 1177.2, 1182.2, 1187.2, 1192.2, 1197.2, 1202.2, 1207.2, 1212.2, 1217.2, 1222.1, 1227.1, 1232.0, 1237.0, 1241.9, 1246.8, 1251.7, 1256.6, 1261.6, 1266.5, 1271.4, 1276.2, 1281.1, 1286.0, 1290.9, 1295.8, 1300.6, 1305.5, 1310.4, 1315.2, 1320.1, 1325.0, 1329.8, 1334.7, 1339.5, 1344.4, 1349.2, 1354.1, 1358.9, 1363.7, 1368.6, 1373.4, 1378.3, 1383.1, 1388.0, 1392.8, 1397.7, 1402.5, 1407.4, 1412.2, 1417.1, 1421.9, 1426.8, 1431.6, 1436.5, 1441.4, 1446.2, 1451.1, 1456.0, 1460.9, 1465.8, 1470.7, 1475.5, 1480.4, 1485.3, 1490.2, 1495.1, 1500.0, 1504.9, 1509.8, 1514.7, 1519.6, 1524.5, 1529.4, 1534.3, 1539.2, 1544.1, 1549.0, 1553.9, 1558.8, 1563.6, 1568.5, 1573.4, 1578.2, 1583.1, 1587.9, 1592.8, 1597.6, 1602.4, 1607.3, 1612.1, 1616.9, 1621.7, 1626.5, 1631.2, 1636.0, 1640.7, 1645.5, 1650.2, 1654.9, 1659.6, 1664.3, 1669.0, 1673.7, 1678.3, 1683.0, 1687.6, 1692.2, 1696.8, 1701.4, 1706.0, 1710.5, 1715.0, 1719.6, 1724.1, 1728.5, 1733.0, 1737.4, 1741.8, 1746.2, 1750.6, 1755.0, 1759.3, 1763.6, 1767.9, 1772.2, 1776.5, 1780.7, 1784.9, 1789.1, 1793.3, 1797.5, 1801.6, 1805.7, 1809.8, 1813.9, 1818.0, 1822.1, 1826.1, 1830.1, 1834.1, 1838.1, 1842.1, 1846.0, 1849.9, 1853.9, 1857.8, 1861.6, 1865.5, 1869.4, 1873.2, 1877.0, 1880.8, 1884.6, 1888.4, 1892.2, 1895.9, 1899.7, 1903.4, 1907.1, 1910.8, 1914.5, 1918.2, 1921.8, 1925.5, 1929.1, 1932.8, 1936.4, 1940.0, 1943.6, 1947.2, 1950.7, 1954.3, 1957.8, 1961.4, 1964.9, 1968.4, 1971.9, 1975.5, 1978.9, 1982.4, 1985.9, 1989.4, 1992.8, 1996.3, 1999.7, 2003.2, 2006.6, 2010.0, 2013.4, 2016.9, 2020.3, 2023.7, 2027.0, 2030.4, 2033.8, 2037.2, 2040.6, 2043.9, 2047.3, 2050.6, 2054.0, 2057.3, 2060.7, 2064.0, 2067.3, 2070.7, 2074.0, 2077.3, 2080.6, 2083.9, 2087.3, 2090.6, 2093.9, 2097.1, 2100.4, 2103.7, 2107.0, 2110.2, 2113.5, 2116.8, 2120.0, 2123.3, 2126.5, 2129.7, 2133.0, 2136.2, 2139.4, 2142.6, 2145.8, 2149.0, 2152.2, 2155.4, 2158.5, 2161.7, 2164.9, 2168.0, 2171.2, 2174.3, 2177.4, 2180.5, 2183.7, 2186.8, 2189.9, 2193.0, 2196.0, 2199.1, 2202.2, 2205.2, 2208.3, 2211.3, 2214.4, 2217.4, 2220.4, 2223.4, 2226.4, 2229.4, 2232.4, 2235.3, 2238.3, 2241.3, 2244.2, 2247.1, 2250.0, 2253.0, 2255.9, 2258.8, 2261.6, 2264.5, 2267.4, 2270.2, 2273.1, 2275.9, 2278.7, 2281.5, 2284.3, 2287.1, 2289.9, 2292.7, 2295.4, 2298.1, 2300.9, 2303.6, 2306.3, 2309.0, 2311.7, 2314.4, 2317.0, 2319.7, 2322.3, 2324.9, 2327.6, 2330.2, 2332.7, 2335.3, 2337.9, 2340.4, 2343.0, 2345.5, 2348.0, 2350.5, 2353.0, 2355.5, 2357.9, 2360.4, 2362.8, 2365.2, 2367.6, 2370.0, 2372.4, 2374.8, 2377.1, 2379.5, 2381.8, 2384.1, 2386.4, 2388.7, 2391.0, 2393.2, 2395.5, 2397.7, 2400.0, 2402.2, 2404.4, 2406.6, 2408.7, 2410.9, 2413.0, 2415.2, 2417.3, 2419.4, 2421.5, 2423.6, 2425.7, 2427.8, 2429.8, 2431.9, 2433.9, 2435.9, 2437.9, 2439.9, 2441.9, 2443.8, 2445.8, 2447.7, 2449.7, 2451.6, 2453.5, 2455.4, 2457.3, 2459.2, 2461.0, 2462.9, 2464.7, 2466.6, 2468.4, 2470.2, 2472.0, 2473.7, 2475.5, 2477.3, 2479.0, 2480.8, 2482.5, 2484.2, 2485.9, 2487.6, 2489.3, 2491.0, 2492.6, 2494.3, 2495.9, 2497.5, 2499.1, 2500.8, 2502.3, 2503.9, 2505.5, 2507.1, 2508.6, 2510.2, 2511.7, 2513.2, 2514.7, 2516.2, 2517.7, 2519.2, 2520.6, 2522.1, 2523.6, 2525.0, 2526.4, 2527.8, 2529.2, 2530.6, 2532.0, 2533.4, 2534.8, 2536.1, 2537.5, 2538.8, 2540.1, 2541.4, 2542.7, 2544.0, 2545.3, 2546.6, 2547.9, 2549.1, 2550.4, 2551.6, 2552.8, 2554.1, 2555.3, 2556.5, 2557.7, 2558.8, 2560.0, 2561.2, 2562.3, 2563.5, 2564.6, 2565.7, 2566.8, 2568.0, 2569.1, 2570.1, 2571.2, 2572.3, 2573.4, 2574.4, 2575.4, 2576.5, 2577.5, 2578.5, 2579.5, 2580.5, 2581.5, 2582.5, 2583.5, 2584.4, 2585.4, 2586.4, 2587.3, 2588.2, 2589.1, 2590.1, 2591.0, 2591.9, 2592.8, 2593.6, 2594.5, 2595.4, 2596.2, 2597.1, 2597.9, 2598.8, 2599.6, 2600.4, 2601.2, 2602.0, 2602.8, 2603.6, 2604.4, 2605.1, 2605.9, 2606.6, 2607.4, 2608.1, 2608.9, 2609.6, 2610.3, 2611.0, 2611.7, 2612.4, 2613.1, 2613.8, 2614.4, 2615.1, 2615.8, 2616.4, 2617.1, 2617.7, 2618.3, 2618.9, 2619.5, 2620.2, 2620.8, 2621.3, 2621.9, 2622.5, 2623.1, 2623.7, 2624.2, 2624.8, 2625.3, 2625.8, 2626.4, 2626.9, 2627.4, 2627.9, 2628.4, 2628.9, 2629.4, 2629.9, 2630.4, 2630.9, 2631.3, 2631.8, 2632.3, 2632.7, 2633.1, 2633.6, 2634.0, 2634.4, 2634.9, 2635.3, 2635.7, 2636.1, 2636.5, 2636.9, 2637.2, 2637.6, 2638.0, 2638.3, 2638.7, 2639.1, 2639.4, 2639.7, 2640.1, 2640.4, 2640.7, 2641.1, 2641.4, 2641.7, 2642.0, 2642.3, 2642.6, 2642.9, 2643.2, 2643.4, 2643.7, 2644.0, 2644.2, 2644.5, 2644.7, 2645.0, 2645.2, 2645.5, 2645.7, 2645.9, 2646.2, 2646.4, 2646.6, 2646.8, 2647.0, 2647.2, 2647.4, 2647.6, 2647.8, 2648.0, 2648.1, 2648.3, 2648.5, 2648.6, 2648.8, 2648.9, 2649.1, 2649.2, 2649.4, 2649.5, 2649.7, 2649.8, 2649.9, 2650.0, 2650.1, 2650.3, 2650.4, 2650.5, 2650.6, 2650.7, 2650.8, 2650.9, 2650.9, 2651.0, 2651.1, 2651.2, 2651.3, 2651.3, 2651.4, 2651.5, 2651.5, 2651.6, 2651.6, 2651.7, 2651.7, 2651.7, 2651.8, 2651.8, 2651.9, 2651.9, 2651.9, 2651.9, 2651.9, 2652.0, 2652.0, 2652.0, 2652.0, 2652.0]
y3=[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.1, 4.6, 5.2, 5.7, 6.3, 6.9, 7.5, 8.1, 8.7, 9.3, 10.0, 10.6, 11.2, 11.9, 12.6, 13.2, 13.9, 14.6, 15.3, 16.0, 16.8, 17.5, 18.3, 19.0, 19.8, 20.6, 21.3, 22.1, 22.9, 23.8, 24.6, 25.4, 26.3, 27.1, 28.0, 28.9, 29.8, 30.7, 31.6, 32.5, 33.4, 34.4, 35.3, 36.3, 37.3, 38.3, 39.3, 40.3, 41.3, 42.3, 43.4, 44.4, 45.5, 46.6, 47.7, 48.8, 49.9, 51.0, 52.1, 53.3, 54.5, 55.6, 56.8, 58.0, 59.2, 60.4, 61.7, 62.9, 64.2, 65.5, 66.7, 68.0, 69.3, 70.7, 72.0, 73.3, 74.7, 76.1, 77.5, 78.9, 80.3, 81.7, 83.1, 84.6, 86.1, 87.5, 89.0, 90.5, 92.0, 93.6, 95.1, 96.7, 98.2, 99.8, 101.4, 103.0, 104.7, 106.3, 108.0, 109.6, 111.3, 113.0, 114.7, 116.4, 118.2, 119.9, 121.7, 123.5, 125.3, 127.1, 128.9, 130.8, 132.6, 134.5, 136.4, 138.3, 140.2, 142.1, 144.0, 146.0, 148.0, 150.0, 152.0, 154.0, 156.0, 158.1, 160.1, 162.2, 164.3, 166.4, 168.6, 170.7, 172.9, 175.0, 177.2, 179.4, 181.7, 183.9, 186.1, 188.4, 190.7, 193.0, 195.3, 197.7, 200.0, 202.4, 204.8, 207.2, 209.6, 212.0, 214.5, 216.9, 219.4, 221.9, 224.4, 227.0, 229.5, 232.1, 234.7, 237.3, 239.9, 242.5, 245.2, 247.9, 250.6, 253.3, 256.0, 258.7, 261.5, 264.3, 267.1, 269.9, 272.7, 275.6, 278.4, 281.3, 284.2, 287.1, 290.1, 293.0, 296.0, 299.0, 302.0, 305.0, 308.1, 311.2, 314.2, 317.4, 320.5, 323.6, 326.8, 330.0, 333.2, 336.4, 339.6, 342.9, 346.2, 349.5, 352.8, 356.1, 359.5, 362.9, 366.3, 369.7, 373.2, 376.6, 380.1, 383.6, 387.1, 390.7, 394.3, 397.9, 401.5, 405.1, 408.8, 412.4, 416.2, 419.9, 423.6, 427.4, 431.2, 435.0, 438.8, 442.7, 446.6, 450.5, 454.4, 458.4, 462.4, 466.4, 470.4, 474.4, 478.5, 482.6, 486.7, 490.9, 495.1, 499.3, 503.5, 507.7, 512.0, 516.3, 520.6, 524.9, 529.3, 533.7, 538.1, 542.6, 547.1, 551.6, 556.1, 560.6, 565.2, 569.8, 574.4, 579.1, 583.8, 588.5, 593.2, 598.0, 602.8, 607.6, 612.5, 617.3, 622.2, 627.1, 632.1, 637.1, 642.1, 647.1, 652.1, 657.2, 662.3, 667.4, 672.5, 677.7, 682.9, 688.0, 693.3, 698.5, 703.8, 709.0, 714.3, 719.6, 725.0, 730.3, 735.7, 741.1, 746.5, 751.9, 757.3, 762.7, 768.2, 773.7, 779.2, 784.7, 790.2, 795.7, 801.2, 806.8, 812.4, 817.9, 823.5, 829.1, 834.7, 840.4, 846.0, 851.6, 857.3, 862.9, 868.6, 874.3, 879.9, 885.6, 891.3, 897.0, 902.7, 908.4, 914.1, 919.9, 925.6, 931.3, 937.1, 942.8, 948.6, 954.3, 960.1, 965.9, 971.7, 977.5, 983.3, 989.2, 995.0, 1000.8, 1006.7, 1012.6, 1018.5, 1024.4, 1030.3, 1036.2, 1042.1, 1048.1, 1054.1, 1060.0, 1066.0, 1072.1, 1078.1, 1084.1, 1090.2, 1096.3, 1102.4, 1108.5, 1114.6, 1120.7, 1126.9, 1133.1, 1139.3, 1145.5, 1151.7, 1158.0, 1164.3, 1170.6, 1176.9, 1183.3, 1189.6, 1196.0, 1202.4, 1208.8, 1215.3, 1221.8, 1228.2, 1234.8, 1241.3, 1247.8, 1254.4, 1261.0, 1267.5, 1274.1, 1280.8, 1287.4, 1294.0, 1300.7, 1307.3, 1314.0, 1320.7, 1327.4, 1334.1, 1340.8, 1347.5, 1354.2, 1360.9, 1367.6, 1374.3, 1381.1, 1387.8, 1394.5, 1401.3, 1408.0, 1414.7, 1421.4, 1428.1, 1434.9, 1441.6, 1448.3, 1455.0, 1461.7, 1468.3, 1475.0, 1481.7, 1488.3, 1495.0, 1501.6, 1508.3, 1514.9, 1521.5, 1528.1, 1534.7, 1541.2, 1547.8, 1554.3, 1560.9, 1567.4, 1574.0, 1580.5, 1587.0, 1593.5, 1600.0, 1606.5, 1613.0, 1619.5, 1626.0, 1632.5, 1639.0, 1645.4, 1651.9, 1658.4, 1664.8, 1671.3, 1677.7, 1684.2, 1690.7, 1697.1, 1703.6, 1710.0, 1716.5, 1722.9, 1729.4, 1735.8, 1742.3, 1748.7, 1755.2, 1761.6, 1768.1, 1774.6, 1781.0, 1787.5, 1794.0, 1800.5, 1807.0, 1813.4, 1819.9, 1826.4, 1832.9, 1839.4, 1845.9, 1852.4, 1858.9, 1865.4, 1871.9, 1878.3, 1884.8, 1891.3, 1897.8, 1904.2, 1910.7, 1917.1, 1923.6, 1930.0, 1936.5, 1942.9, 1949.3, 1955.7, 1962.1, 1968.5, 1974.9, 1981.2, 1987.6, 1993.9, 2000.2, 2006.5, 2012.8, 2019.1, 2025.4, 2031.6, 2037.8, 2044.0, 2050.2, 2056.4, 2062.5, 2068.7, 2074.8, 2080.9, 2086.9, 2093.0, 2099.0, 2105.0, 2111.0, 2116.9, 2122.9, 2128.8, 2134.7, 2140.5, 2146.4, 2152.2, 2158.0, 2163.8, 2169.6, 2175.4, 2181.1, 2186.9, 2192.6, 2198.3, 2204.0, 2209.6, 2215.3, 2220.9, 2226.6, 2232.2, 2237.8, 2243.4, 2249.0, 2254.6, 2260.1, 2265.7, 2271.2, 2276.8, 2282.3, 2287.8, 2293.3, 2298.9, 2304.4, 2309.9, 2315.4, 2320.8, 2326.3, 2331.8, 2337.3, 2342.8, 2348.2, 2353.7, 2359.2, 2364.6, 2370.1, 2375.6, 2381.0, 2386.5, 2392.0, 2397.5, 2402.9, 2408.4, 2413.9, 2419.4, 2424.8, 2430.3, 2435.8, 2441.3, 2446.7, 2452.2, 2457.7, 2463.2, 2468.6, 2474.1, 2479.5, 2485.0, 2490.5, 2495.9, 2501.3, 2506.8, 2512.2, 2517.6, 2523.1, 2528.5, 2533.9, 2539.3, 2544.7, 2550.0, 2555.4, 2560.8, 2566.1, 2571.5, 2576.8, 2582.1, 2587.5, 2592.8, 2598.1, 2603.3, 2608.6, 2613.9, 2619.1, 2624.3, 2629.6, 2634.8, 2639.9, 2645.1, 2650.3, 2655.4, 2660.6, 2665.7, 2670.8, 2675.8, 2680.9, 2686.0, 2691.0, 2696.0, 2701.0, 2706.0, 2710.9, 2715.8, 2720.8, 2725.7, 2730.6, 2735.4, 2740.3, 2745.1, 2749.9, 2754.7, 2759.5, 2764.3, 2769.0, 2773.7, 2778.4, 2783.1, 2787.8, 2792.5, 2797.1, 2801.7, 2806.3, 2810.9, 2815.5, 2820.1, 2824.6, 2829.2, 2833.7, 2838.2, 2842.6, 2847.1, 2851.6, 2856.0, 2860.4, 2864.8, 2869.2, 2873.6, 2877.9, 2882.3, 2886.6, 2890.9, 2895.2, 2899.5, 2903.7, 2908.0, 2912.2, 2916.5, 2920.7, 2924.8, 2929.0, 2933.2, 2937.3, 2941.5, 2945.6, 2949.7, 2953.8, 2957.9, 2961.9, 2966.0, 2970.0, 2974.0, 2978.0, 2982.0, 2986.0, 2990.0, 2993.9, 2997.9, 3001.8, 3005.7, 3009.6, 3013.5, 3017.4, 3021.2, 3025.1, 3028.9, 3032.7, 3036.6, 3040.3, 3044.1, 3047.9, 3051.7, 3055.4, 3059.1, 3062.8, 3066.5, 3070.2, 3073.9, 3077.6, 3081.2, 3084.9, 3088.5, 3092.1, 3095.7, 3099.3, 3102.9, 3106.4, 3110.0, 3113.5, 3117.0, 3120.5, 3124.0, 3127.5, 3131.0, 3134.5, 3137.9, 3141.3, 3144.8, 3148.2, 3151.6, 3155.0, 3158.3, 3161.7, 3165.0, 3168.4, 3171.7, 3175.0, 3178.3, 3181.6, 3184.9, 3188.1, 3191.4, 3194.6, 3197.8, 3201.0, 3204.2, 3207.4, 3210.6, 3213.8, 3216.9, 3220.1, 3223.2, 3226.3, 3229.4, 3232.5, 3235.6, 3238.7, 3241.7, 3244.7, 3247.8, 3250.8, 3253.8, 3256.8, 3259.8, 3262.8, 3265.7, 3268.7, 3271.6, 3274.5, 3277.4, 3280.3, 3283.2, 3286.1, 3289.0, 3291.8, 3294.7, 3297.5, 3300.3, 3303.1, 3305.9, 3308.7, 3311.5, 3314.3, 3317.0, 3319.8, 3322.5, 3325.2, 3327.9, 3330.6, 3333.3, 3335.9, 3338.6, 3341.2, 3343.9, 3346.5, 3349.1, 3351.7, 3354.3, 3356.9, 3359.4, 3362.0, 3364.5, 3367.0, 3369.5, 3372.0, 3374.5, 3377.0, 3379.5, 3381.9, 3384.4, 3386.8, 3389.2, 3391.6, 3394.0, 3396.4, 3398.7, 3401.1, 3403.4, 3405.8, 3408.1, 3410.4, 3412.7, 3415.0, 3417.3, 3419.5, 3421.8, 3424.0, 3426.2, 3428.4, 3430.6, 3432.8, 3435.0, 3437.1, 3439.3, 3441.4, 3443.5, 3445.6, 3447.7, 3449.8, 3451.9, 3453.9, 3456.0, 3458.0, 3460.1, 3462.1, 3464.1, 3466.0, 3468.0, 3470.0, 3471.9, 3473.8, 3475.8, 3477.7, 3479.6, 3481.5, 3483.3, 3485.2, 3487.0, 3488.9, 3490.7, 3492.5, 3494.3, 3496.1, 3497.8, 3499.6, 3501.3, 3503.0, 3504.8, 3506.5, 3508.2, 3509.8, 3511.5, 3513.2, 3514.8, 3516.4, 3518.0, 3519.6, 3521.2, 3522.8, 3524.3, 3525.9, 3527.4, 3528.9, 3530.5, 3532.0, 3533.4, 3534.9, 3536.4, 3537.8, 3539.2, 3540.6, 3542.0, 3543.4, 3544.8, 3546.2, 3547.5, 3548.9, 3550.2, 3551.5, 3552.8, 3554.1, 3555.3, 3556.6, 3557.8, 3559.1, 3560.3, 3561.5, 3562.7, 3563.9, 3565.0, 3566.2, 3567.3, 3568.4, 3569.5, 3570.6, 3571.7, 3572.8, 3573.8, 3574.9, 3575.9, 3576.9, 3577.9, 3578.9, 3579.9, 3580.8, 3581.8, 3582.7, 3583.6, 3584.5, 3585.4, 3586.3, 3587.1, 3588.0, 3588.8, 3589.6, 3590.5, 3591.2, 3592.0, 3592.8, 3593.5, 3594.3, 3595.0, 3595.7, 3596.4, 3597.1, 3597.8, 3598.4, 3599.1, 3599.7, 3600.3, 3600.9, 3601.5, 3602.1, 3602.7, 3603.2, 3603.8, 3604.3, 3604.8, 3605.3, 3605.8, 3606.3, 3606.7, 3607.2, 3607.6, 3608.0, 3608.4, 3608.8, 3609.2, 3609.6, 3610.0, 3610.3, 3610.6, 3611.0, 3611.3, 3611.6, 3611.8, 3612.1, 3612.4, 3612.6, 3612.9, 3613.1, 3613.3, 3613.5, 3613.7, 3613.8, 3614.0, 3614.1, 3614.3, 3614.4, 3614.5, 3614.6, 3614.7, 3614.8, 3614.9, 3614.9, 3614.9, 3615.0, 3615.0]
y4=[0.0, 0.6, 1.2, 1.9, 2.7, 3.5, 4.4, 5.3, 6.2, 7.2, 8.3, 9.4, 10.5, 11.7, 12.9, 14.2, 15.5, 16.9, 18.3, 19.7, 21.2, 22.7, 24.3, 25.9, 27.5, 29.2, 30.9, 32.6, 34.4, 36.2, 38.0, 39.9, 41.8, 43.7, 45.6, 47.6, 49.6, 51.7, 53.7, 55.8, 57.9, 60.1, 62.2, 64.4, 66.6, 68.8, 71.1, 73.3, 75.6, 77.9, 80.2, 82.5, 84.9, 87.2, 89.6, 92.0, 94.4, 96.8, 99.2, 101.6, 104.0, 106.5, 108.9, 111.4, 113.8, 116.3, 118.8, 121.3, 123.7, 126.2, 128.7, 131.2, 133.6, 136.1, 138.6, 141.1, 143.5, 146.0, 148.5, 150.9, 153.4, 155.8, 158.3, 160.7, 163.1, 165.6, 168.0, 170.4, 172.9, 175.3, 177.7, 180.1, 182.6, 185.0, 187.4, 189.9, 192.3, 194.7, 197.2, 199.6, 202.1, 204.5, 206.9, 209.4, 211.9, 214.3, 216.8, 219.3, 221.8, 224.2, 226.7, 229.2, 231.8, 234.3, 236.8, 239.3, 241.9, 244.4, 247.0, 249.6, 252.2, 254.8, 257.4, 260.0, 262.6, 265.3, 267.9, 270.6, 273.3, 276.0, 278.7, 281.4, 284.2, 286.9, 289.7, 292.5, 295.3, 298.1, 301.0, 303.8, 306.7, 309.6, 312.5, 315.4, 318.4, 321.4, 324.4, 327.4, 330.4, 333.5, 336.6, 339.7, 342.8, 345.9, 349.1, 352.3, 355.5, 358.8, 362.0, 365.3, 368.7, 372.0, 375.4, 378.8, 382.2, 385.7, 389.1, 392.7, 396.2, 399.8, 403.4, 407.0, 410.7, 414.3, 418.1, 421.8, 425.6, 429.4, 433.3, 437.1, 441.1, 445.0, 449.0, 453.0, 457.0, 461.1, 465.2, 469.4, 473.6, 477.8, 482.0, 486.3, 490.6, 494.9, 499.3, 503.7, 508.1, 512.6, 517.0, 521.6, 526.1, 530.7, 535.3, 539.9, 544.5, 549.2, 553.9, 558.7, 563.4, 568.2, 573.0, 577.8, 582.7, 587.6, 592.5, 597.4, 602.4, 607.4, 612.4, 617.4, 622.4, 627.5, 632.6, 637.7, 642.8, 648.0, 653.2, 658.4, 663.6, 668.8, 674.1, 679.3, 684.6, 689.9, 695.3, 700.6, 706.0, 711.4, 716.8, 722.2, 727.6, 733.1, 738.5, 744.0, 749.5, 755.0, 760.5, 766.1, 771.6, 777.2, 782.8, 788.4, 794.1, 799.7, 805.4, 811.1, 816.8, 822.6, 828.4, 834.2, 840.0, 845.8, 851.7, 857.6, 863.5, 869.5, 875.5, 881.5, 887.5, 893.6, 899.7, 905.8, 912.0, 918.2, 924.4, 930.7, 937.0, 943.3, 949.7, 956.1, 962.5, 969.0, 975.5, 982.1, 988.7, 995.3, 1002.0, 1008.8, 1015.5, 1022.3, 1029.2, 1036.1, 1043.0, 1050.0, 1057.0, 1064.1, 1071.2, 1078.3, 1085.5, 1092.7, 1100.0, 1107.3, 1114.6, 1122.0, 1129.4, 1136.8, 1144.2, 1151.7, 1159.1, 1166.6, 1174.2, 1181.7, 1189.3, 1196.9, 1204.5, 1212.1, 1219.7, 1227.3, 1234.9, 1242.6, 1250.2, 1257.9, 1265.6, 1273.2, 1280.9, 1288.5, 1296.2, 1303.9, 1311.5, 1319.1, 1326.8, 1334.4, 1342.0, 1349.6, 1357.2, 1364.8, 1372.3, 1379.9, 1387.4, 1394.9, 1402.4, 1409.9, 1417.4, 1424.8, 1432.3, 1439.7, 1447.1, 1454.5, 1461.9, 1469.3, 1476.7, 1484.0, 1491.3, 1498.6, 1505.9, 1513.2, 1520.4, 1527.7, 1534.9, 1542.1, 1549.3, 1556.5, 1563.6, 1570.7, 1577.8, 1584.9, 1592.0, 1599.1, 1606.1, 1613.1, 1620.1, 1627.1, 1634.1, 1641.0, 1647.9, 1654.8, 1661.7, 1668.6, 1675.4, 1682.3, 1689.1, 1696.0, 1702.8, 1709.6, 1716.4, 1723.3, 1730.1, 1737.0, 1743.8, 1750.7, 1757.6, 1764.5, 1771.4, 1778.3, 1785.3, 1792.2, 1799.2, 1806.3, 1813.3, 1820.4, 1827.6, 1834.7, 1841.9, 1849.2, 1856.4, 1863.8, 1871.2, 1878.6, 1886.1, 1893.6, 1901.2, 1908.8, 1916.5, 1924.3, 1932.1, 1940.0, 1948.0, 1956.0, 1964.1, 1972.2, 1980.4, 1988.6, 1996.9, 2005.2, 2013.6, 2022.0, 2030.4, 2038.9, 2047.3, 2055.8, 2064.3, 2072.8, 2081.4, 2089.9, 2098.4, 2106.9, 2115.4, 2123.9, 2132.3, 2140.8, 2149.2, 2157.6, 2165.9, 2174.3, 2182.5, 2190.8, 2199.0, 2207.1, 2215.2, 2223.2, 2231.1, 2239.0, 2246.8, 2254.5, 2262.2, 2269.8, 2277.3, 2284.8, 2292.2, 2299.6, 2306.8, 2314.1, 2321.3, 2328.4, 2335.4, 2342.5, 2349.4, 2356.4, 2363.2, 2370.1, 2376.9, 2383.6, 2390.3, 2397.0, 2403.7, 2410.3, 2416.8, 2423.4, 2429.9, 2436.4, 2442.9, 2449.3, 2455.7, 2462.1, 2468.5, 2474.9, 2481.2, 2487.6, 2493.9, 2500.2, 2506.5, 2512.8, 2519.1, 2525.4, 2531.7, 2538.0, 2544.3, 2550.6, 2556.9, 2563.2, 2569.5, 2575.8, 2582.1, 2588.4, 2594.6, 2600.9, 2607.2, 2613.5, 2619.8, 2626.0, 2632.3, 2638.5, 2644.8, 2651.0, 2657.3, 2663.5, 2669.7, 2675.9, 2682.1, 2688.2, 2694.4, 2700.6, 2706.7, 2712.8, 2718.9, 2725.0, 2731.1, 2737.1, 2743.2, 2749.2, 2755.2, 2761.2, 2767.2, 2773.1, 2779.0, 2784.9, 2790.8, 2796.7, 2802.5, 2808.3, 2814.1, 2819.9, 2825.6, 2831.3, 2837.0, 2842.6, 2848.3, 2853.9, 2859.4, 2865.0, 2870.5, 2876.0, 2881.5, 2886.9, 2892.3, 2897.7, 2903.1, 2908.5, 2913.8, 2919.1, 2924.4, 2929.7, 2934.9, 2940.1, 2945.3, 2950.5, 2955.7, 2960.8, 2965.9, 2971.0, 2976.1, 2981.2, 2986.2, 2991.2, 2996.3, 3001.3, 3006.2, 3011.2, 3016.2, 3021.1, 3026.0, 3030.9, 3035.8, 3040.7, 3045.6, 3050.4, 3055.2, 3060.1, 3064.9, 3069.7, 3074.5, 3079.3, 3084.0, 3088.8, 3093.6, 3098.3, 3103.0, 3107.8, 3112.5, 3117.2, 3121.9, 3126.6, 3131.3, 3136.0, 3140.7, 3145.3, 3150.0, 3154.7, 3159.3, 3164.0, 3168.6, 3173.2, 3177.9, 3182.5, 3187.1, 3191.7, 3196.3, 3200.9, 3205.5, 3210.0, 3214.6, 3219.1, 3223.7, 3228.2, 3232.7, 3237.2, 3241.8, 3246.2, 3250.7, 3255.2, 3259.7, 3264.1, 3268.6, 3273.0, 3277.4, 3281.8, 3286.2, 3290.6, 3295.0, 3299.3, 3303.7, 3308.0, 3312.3, 3316.6, 3320.9, 3325.2, 3329.5, 3333.7, 3337.9, 3342.2, 3346.4, 3350.6, 3354.8, 3358.9, 3363.1, 3367.2, 3371.3, 3375.4, 3379.5, 3383.6, 3387.6, 3391.7, 3395.7, 3399.7, 3403.7, 3407.7, 3411.6, 3415.6, 3419.5, 3423.4, 3427.3, 3431.1, 3435.0, 3438.8, 3442.6, 3446.4, 3450.2, 3453.9, 3457.7, 3461.4, 3465.1, 3468.8, 3472.4, 3476.1, 3479.7, 3483.3, 3486.9, 3490.5, 3494.0, 3497.6, 3501.1, 3504.6, 3508.1, 3511.6, 3515.0, 3518.5, 3521.9, 3525.3, 3528.7, 3532.1, 3535.4, 3538.8, 3542.1, 3545.4, 3548.7, 3552.0, 3555.3, 3558.5, 3561.8, 3565.0, 3568.2, 3571.4, 3574.5, 3577.7, 3580.9, 3584.0, 3587.1, 3590.2, 3593.3, 3596.4, 3599.4, 3602.5, 3605.5, 3608.5, 3611.5, 3614.5, 3617.5, 3620.4, 3623.4, 3626.3, 3629.2, 3632.2, 3635.1, 3637.9, 3640.8, 3643.7, 3646.5, 3649.3, 3652.2, 3655.0, 3657.8, 3660.6, 3663.3, 3666.1, 3668.8, 3671.6, 3674.3, 3677.0, 3679.7, 3682.4, 3685.1, 3687.8, 3690.4, 3693.1, 3695.7, 3698.3, 3700.9, 3703.5, 3706.1, 3708.7, 3711.3, 3713.8, 3716.4, 3718.9, 3721.5, 3724.0, 3726.5, 3729.0, 3731.5, 3734.0, 3736.5, 3738.9, 3741.4, 3743.8, 3746.3, 3748.7, 3751.1, 3753.5, 3755.9, 3758.3, 3760.7, 3763.1, 3765.4, 3767.8, 3770.1, 3772.5, 3774.8, 3777.1, 3779.4, 3781.7, 3784.0, 3786.3, 3788.5, 3790.8, 3793.0, 3795.3, 3797.5, 3799.7, 3801.9, 3804.1, 3806.3, 3808.5, 3810.7, 3812.8, 3815.0, 3817.1, 3819.2, 3821.4, 3823.5, 3825.6, 3827.7, 3829.8, 3831.8, 3833.9, 3835.9, 3838.0, 3840.0, 3842.0, 3844.1, 3846.1, 3848.1, 3850.0, 3852.0, 3854.0, 3855.9, 3857.9, 3859.8, 3861.7, 3863.6, 3865.5, 3867.4, 3869.3, 3871.2, 3873.1, 3874.9, 3876.8, 3878.6, 3880.4, 3882.2, 3884.0, 3885.8, 3887.6, 3889.4, 3891.1, 3892.9, 3894.6, 3896.4, 3898.1, 3899.8, 3901.5, 3903.2, 3904.9, 3906.5, 3908.2, 3909.8, 3911.5, 3913.1, 3914.7, 3916.3, 3917.9, 3919.5, 3921.1, 3922.6, 3924.2, 3925.7, 3927.3, 3928.8, 3930.3, 3931.8, 3933.3, 3934.8, 3936.3, 3937.7, 3939.2, 3940.6, 3942.0, 3943.5, 3944.9, 3946.3, 3947.7, 3949.0, 3950.4, 3951.7, 3953.1, 3954.4, 3955.7, 3957.1, 3958.4, 3959.6, 3960.9, 3962.2, 3963.4, 3964.7, 3965.9, 3967.2, 3968.4, 3969.6, 3970.8, 3971.9, 3973.1, 3974.3, 3975.4, 3976.6, 3977.7, 3978.8, 3979.9, 3981.0, 3982.1, 3983.2, 3984.2, 3985.3, 3986.3, 3987.3, 3988.3, 3989.4, 3990.3, 3991.3, 3992.3, 3993.3, 3994.2, 3995.2, 3996.1, 3997.0, 3997.9, 3998.8, 3999.7, 4000.5, 4001.4, 4002.3, 4003.1, 4003.9, 4004.7, 4005.5, 4006.3, 4007.1, 4007.9, 4008.6, 4009.4, 4010.1, 4010.8, 4011.6, 4012.3, 4013.0, 4013.6, 4014.3, 4015.0, 4015.6, 4016.2, 4016.9, 4017.5, 4018.1, 4018.6, 4019.2, 4019.8, 4020.3, 4020.9, 4021.4, 4021.9, 4022.4, 4022.9, 4023.4, 4023.9, 4024.3, 4024.8, 4025.2, 4025.6, 4026.1, 4026.5, 4026.8, 4027.2, 4027.6, 4027.9, 4028.3, 4028.6, 4028.9, 4029.2, 4029.5, 4029.8, 4030.1, 4030.3, 4030.6, 4030.8, 4031.0, 4031.3, 4031.5, 4031.6, 4031.8, 4032.0, 4032.1, 4032.3, 4032.4, 4032.5, 4032.6, 4032.7, 4032.8, 4032.8, 4032.9, 4032.9, 4033.0, 4033.0]
yr=[0.0, 0.1, 0.2, 0.4, 0.5, 0.6, 0.8, 0.9, 1.1, 1.2, 1.4, 1.5, 1.7, 1.9, 2.0, 2.2, 2.4, 2.6, 2.7, 2.9, 3.1, 3.3, 3.5, 3.7, 3.9, 4.1, 4.3, 4.5, 4.7, 5.0, 5.2, 5.4, 5.6, 5.9, 6.1, 6.3, 6.6, 6.8, 7.1, 7.3, 7.6, 7.8, 8.1, 8.4, 8.6, 8.9, 9.2, 9.4, 9.7, 10.0, 10.3, 10.6, 10.9, 11.2, 11.5, 11.8, 12.1, 12.4, 12.7, 13.0, 13.3, 13.6, 13.9, 14.3, 14.6, 14.9, 15.2, 15.6, 15.9, 16.2, 16.6, 16.9, 17.3, 17.6, 18.0, 18.3, 18.7, 19.1, 19.4, 19.8, 20.2, 20.5, 20.9, 21.3, 21.7, 22.0, 22.4, 22.8, 23.2, 23.6, 24.0, 24.4, 24.8, 25.2, 25.6, 26.0, 26.4, 26.8, 27.2, 27.7, 28.1, 28.5, 28.9, 29.4, 29.8, 30.2, 30.6, 31.1, 31.5, 32.0, 32.4, 32.8, 33.3, 33.7, 34.2, 34.7, 35.1, 35.6, 36.0, 36.5, 37.0, 37.4, 37.9, 38.4, 38.8, 39.3, 39.8, 40.3, 40.7, 41.2, 41.7, 42.2, 42.7, 43.2, 43.7, 44.2, 44.7, 45.2, 45.7, 46.2, 46.7, 47.2, 47.7, 48.2, 48.7, 49.2, 49.8, 50.3, 50.8, 51.3, 51.8, 52.4, 52.9, 53.4, 54.0, 54.5, 55.0, 55.6, 56.1, 56.6, 57.2, 57.7, 58.3, 58.8, 59.4, 59.9, 60.5, 61.0, 61.6, 62.1, 62.7, 63.2, 63.8, 64.4, 64.9, 65.5, 66.0, 66.6, 67.2, 67.8, 68.3, 68.9, 69.5, 70.1, 70.6, 71.2, 71.8, 72.4, 73.0, 73.5, 74.1, 74.7, 75.3, 75.9, 76.5, 77.1, 77.7, 78.3, 78.8, 79.4, 80.0, 80.6, 81.2, 81.8, 82.4, 83.0, 83.7, 84.3, 84.9, 85.5, 86.1, 86.7, 87.3, 87.9, 88.5, 89.1, 89.8, 90.4, 91.0, 91.6, 92.2, 92.8, 93.5, 94.1, 94.7, 95.3, 95.9, 96.6, 97.2, 97.8, 98.4, 99.1, 99.7, 100.3, 101.0, 101.6, 102.2, 102.9, 103.5, 104.1, 104.7, 105.4, 106.0, 106.7, 107.3, 107.9, 108.6, 109.2, 109.8, 110.5, 111.1, 111.7, 112.4, 113.0, 113.7, 114.3, 114.9, 115.6, 116.2, 116.9, 117.5, 118.2, 118.8, 119.4, 120.1, 120.7, 121.4, 122.0, 122.7, 123.3, 123.9, 124.6, 125.2, 125.9, 126.5, 127.2, 127.8, 128.5, 129.1, 129.8, 130.4, 131.0, 131.7, 132.3, 133.0, 133.6, 134.3, 134.9, 135.6, 136.2, 136.8, 137.5, 138.1, 138.8, 139.4, 140.1, 140.7, 141.4, 142.0, 142.6, 143.3, 143.9, 144.6, 145.2, 145.9, 146.5, 147.1, 147.8, 148.4, 149.1, 149.7, 150.3, 151.0, 151.6, 152.3, 152.9, 153.5, 154.2, 154.8, 155.4, 156.1, 156.7, 157.4, 158.0, 158.6, 159.3, 159.9, 160.5, 161.2, 161.8, 162.4, 163.1, 163.7, 164.3, 164.9, 165.6, 166.2, 166.8, 167.5, 168.1, 168.7, 169.4, 170.0, 170.6, 171.2, 171.9, 172.5, 173.1, 173.7, 174.4, 175.0, 175.6, 176.2, 176.8, 177.5, 178.1, 178.7, 179.3, 179.9, 180.6, 181.2, 181.8, 182.4, 183.0, 183.6, 184.3, 184.9, 185.5, 186.1, 186.7, 187.3, 187.9, 188.5, 189.1, 189.8, 190.4, 191.0, 191.6, 192.2, 192.8, 193.4, 194.0, 194.6, 195.2, 195.8, 196.4, 197.0, 197.6, 198.2, 198.8, 199.4, 200.0, 200.6, 201.2, 201.8, 202.4, 203.0, 203.6, 204.1, 204.7, 205.3, 205.9, 206.5, 207.1, 207.7, 208.3, 208.8, 209.4, 210.0, 210.6, 211.2, 211.8, 212.3, 212.9, 213.5, 214.1, 214.6, 215.2, 215.8, 216.4, 216.9, 217.5, 218.1, 218.7, 219.2, 219.8, 220.4, 220.9, 221.5, 222.0, 222.6, 223.2, 223.7, 224.3, 224.9, 225.4, 226.0, 226.5, 227.1, 227.6, 228.2, 228.7, 229.3, 229.8, 230.4, 230.9, 231.5, 232.0, 232.6, 233.1, 233.6, 234.2, 234.7, 235.3, 235.8, 236.3, 236.9, 237.4, 237.9, 238.5, 239.0, 239.5, 240.1, 240.6, 241.1, 241.6, 242.2, 242.7, 243.2, 243.7, 244.3, 244.8, 245.3, 245.8, 246.3, 246.8, 247.3, 247.8, 248.4, 248.9, 249.4, 249.9, 250.4, 250.9, 251.4, 251.9, 252.4, 252.9, 253.4, 253.9, 254.4, 254.9, 255.3, 255.8, 256.3, 256.8, 257.3, 257.8, 258.3, 258.7, 259.2, 259.7, 260.2, 260.6, 261.1, 261.6, 262.1, 262.5, 263.0, 263.5, 263.9, 264.4, 264.8, 265.3, 265.8, 266.2, 266.7, 267.1, 267.6, 268.0, 268.5, 268.9, 269.4, 269.8, 270.2, 270.7, 271.1, 271.6, 272.0, 272.4, 272.9, 273.3, 273.7, 274.1, 274.6, 275.0, 275.4, 275.8, 276.2, 276.7, 277.1, 277.5, 277.9, 278.3, 278.7, 279.1, 279.5, 279.9, 280.3, 280.7, 281.1, 281.5, 281.9, 282.3, 282.7, 283.1, 283.5, 283.8, 284.2, 284.6, 285.0, 285.4, 285.7, 286.1, 286.5, 286.8, 287.2, 287.6, 287.9, 288.3, 288.7, 289.0, 289.4, 289.7, 290.1, 290.4, 290.8, 291.1, 291.5, 291.8, 292.1, 292.5, 292.8, 293.1, 293.5, 293.8, 294.1, 294.5, 294.8, 295.1, 295.4, 295.7, 296.0, 296.4, 296.7, 297.0, 297.3, 297.6, 297.9, 298.2, 298.5, 298.8, 299.1, 299.3, 299.6, 299.9, 300.2, 300.5, 300.8, 301.0, 301.3, 301.6, 301.9, 302.1, 302.4, 302.7, 302.9, 303.2, 303.4, 303.7, 303.9, 304.2, 304.4, 304.7, 304.9, 305.2, 305.4, 305.7, 305.9, 306.1, 306.4, 306.6, 306.8, 307.1, 307.3, 307.5, 307.7, 308.0, 308.2, 308.4, 308.6, 308.8, 309.0, 309.2, 309.5, 309.7, 309.9, 310.1, 310.3, 310.5, 310.7, 310.9, 311.1, 311.3, 311.4, 311.6, 311.8, 312.0, 312.2, 312.4, 312.6, 312.7, 312.9, 313.1, 313.3, 313.4, 313.6, 313.8, 314.0, 314.1, 314.3, 314.5, 314.6, 314.8, 314.9, 315.1, 315.3, 315.4, 315.6, 315.7, 315.9, 316.0, 316.2, 316.3, 316.5, 316.6, 316.7, 316.9, 317.0, 317.2, 317.3, 317.4, 317.6, 317.7, 317.9, 318.0, 318.1, 318.2, 318.4, 318.5, 318.6, 318.8, 318.9, 319.0, 319.1, 319.2, 319.4, 319.5, 319.6, 319.7, 319.8, 319.9, 320.1, 320.2, 320.3, 320.4, 320.5, 320.6, 320.7, 320.8, 320.9, 321.0, 321.1, 321.2, 321.3, 321.4, 321.5, 321.6, 321.7, 321.8, 321.9, 322.0, 322.1, 322.2, 322.3, 322.4, 322.5, 322.6, 322.7, 322.8, 322.8, 322.9, 323.0, 323.1, 323.2, 323.3, 323.4, 323.4, 323.5, 323.6, 323.7, 323.8, 323.8, 323.9, 324.0, 324.1, 324.2, 324.2, 324.3, 324.4, 324.5, 324.5, 324.6, 324.7, 324.8, 324.8, 324.9, 325.0, 325.1, 325.1, 325.2, 325.3, 325.3, 325.4, 325.5, 325.5, 325.6, 325.7, 325.7, 325.8, 325.9, 325.9, 326.0, 326.1, 326.1, 326.2, 326.2, 326.3, 326.4, 326.4, 326.5, 326.5, 326.6, 326.7, 326.7, 326.8, 326.8, 326.9, 326.9, 327.0, 327.0, 327.1, 327.1, 327.2, 327.3, 327.3, 327.4, 327.4, 327.5, 327.5, 327.6, 327.6, 327.7, 327.7, 327.7, 327.8, 327.8, 327.9, 327.9, 328.0, 328.0, 328.1, 328.1, 328.2, 328.2, 328.2, 328.3, 328.3, 328.4, 328.4, 328.5, 328.5, 328.5, 328.6, 328.6, 328.6, 328.7, 328.7, 328.8, 328.8, 328.8, 328.9, 328.9, 328.9, 329.0, 329.0, 329.0, 329.1, 329.1, 329.1, 329.2, 329.2, 329.2, 329.3, 329.3, 329.3, 329.4, 329.4, 329.4, 329.5, 329.5, 329.5, 329.5, 329.6, 329.6, 329.6, 329.6, 329.7, 329.7, 329.7, 329.8, 329.8, 329.8, 329.8, 329.9, 329.9, 329.9, 329.9, 329.9, 330.0, 330.0, 330.0, 330.0, 330.1, 330.1, 330.1, 330.1, 330.1, 330.2, 330.2, 330.2, 330.2, 330.2, 330.3, 330.3, 330.3, 330.3, 330.3, 330.3, 330.4, 330.4, 330.4, 330.4, 330.4, 330.4, 330.4, 330.5, 330.5, 330.5, 330.5, 330.5, 330.5, 330.5, 330.6, 330.6, 330.6, 330.6, 330.6, 330.6, 330.6, 330.6, 330.7, 330.7, 330.7, 330.7, 330.7, 330.7, 330.7, 330.7, 330.7, 330.7, 330.8, 330.8, 330.8, 330.8, 330.8, 330.8, 330.8, 330.8, 330.8, 330.8, 330.8, 330.8, 330.8, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 330.9, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0, 331.0]
cs_1=CubicSpline(x,y1) #barra 1
cs_2=CubicSpline(x,y2) #barra 2
cs_3=CubicSpline(x,y3) #barra 3
cs_4=CubicSpline(x,y4) #barra 4
cs_r=CubicSpline(x,yr) #barra reguladora

In [35]:
def k(z, criticality): #z are the readings from CARAPAU
    assert type(z) == list and len(z) == 5 #### determines the value of k com base das posições das barras
    for i in range(0, len(z)):
        z[i]=z[i]*20 #voltages readings from 0-5V, multiply by 20 to give in % reading
    p = (cs_1(z[0]) + cs_2(z[1]) + cs_3(z[2]) + cs_4(z[3]) + cs_r(z[4]) - criticality) * 10**-5 #sum of all rods reactivity (cubic splines give the reactivity for a certain % of insertion) minus criticality (given in the paper) and then converted from pcm
    return -1/(p-1) #this is equal to k

In [36]:
def polinomial_solver(t, pol): #### determines the value of the polinomial (pol) at that instance of time equals t
    assert type(pol)==list
    return sum(list(pol[i]*t**i for i in range(0,len(pol))))

In [37]:
def coefficients_solver(initial_values,time1,p): ### Using Sochaki-Parker using polinomials to get results
    start_time = time.perf_counter()
    a=[initial_values[0]]
    b=[initial_values[1]]
    c=[initial_values[2]]
    d=[initial_values[3]]
    e=[initial_values[4]]
    f=[initial_values[5]]
    g=[initial_values[6]]
    while (abs(polinomial_solver(time1,a)-polinomial_solver(time1,a[:-1]))/abs(polinomial_solver(time1,a)))>3*10**-4:
        a=a+[(1/(len(a)))*(((p-sum(beta))/l)*a[-1]+Lambda[0]*b[-1]+Lambda[1]*c[-1]+Lambda[2]*d[-1]+Lambda[3]*e[-1]+Lambda[4]*f[-1]+Lambda[5]*g[-1])]
        b=b+[(1/(len(b)))*(((beta[0]/l)*a[-2])-Lambda[0]*b[-1])]
        c=c+[(1/(len(c)))*(((beta[1]/l)*a[-2])-Lambda[1]*c[-1])]
        d=d+[(1/(len(d)))*(((beta[2]/l)*a[-2])-Lambda[2]*d[-1])]
        e=e+[(1/(len(e)))*(((beta[3]/l)*a[-2])-Lambda[3]*e[-1])]
        f=f+[(1/(len(f)))*(((beta[4]/l)*a[-2])-Lambda[4]*f[-1])]
        g=g+[(1/(len(g)))*(((beta[5]/l)*a[-2])-Lambda[5]*g[-1])]
    #print([polinomial_solver(time1,a),polinomial_solver(time1,b),polinomial_solver(time1,c),polinomial_solver(time1,d),polinomial_solver(time1,e),polinomial_solver(time1,f),polinomial_solver(time1,g),abs(time.perf_counter()-start_time)])
    return [polinomial_solver(time1,a),polinomial_solver(time1,b),polinomial_solver(time1,c),polinomial_solver(time1,d),polinomial_solver(time1,e),polinomial_solver(time1,f),polinomial_solver(time1,g),abs(time.perf_counter()-start_time)]


In [38]:
def place(counts, k, source): 
    return math.log(1-counts*(1-k)/(source))/math.log(k) #operação inversa da soma geométrica para descobrir o N no expoente correspondente ao numero de contagens
                                                         #dois logs apenas para mudar de base


In [39]:
def continuation(counts, k, source, time):
    if round(counts,8)==round(source/(1-k),8):
        return counts
    elif round(counts,8)>source/(1-k):
        print(True)
        return counts*math.exp((k-1)*time/(ld*10)) #one group approximation para resolver problemas devido a ruído
    else:
        n=place(counts,k,source)
        return source*(1-k**(n+time/(ld)))/(1-k) #nest iteration of the geometric sum


In [48]:
def instruments():
    instrument.write('SOUR1:FUNC PULS') #modo pulso
    instrument.write('SOUR1:BURS:STATe OFF') #modo burst off
    instrument.write('SOUR1:FREQ 2Hz') 
    instrument.write('SOUR1:PULS:WIDTH 1000ns')
    instrument.write('OUTP1:POL NORM') #pulso positivo apenas
    instrument.write('SOUR1:VOLT:LEV:IMM:AMPL 5VPP') #max 5V
    instrument.write('SOUR1:VOLT:LEV:IMM:OFFS 2.5V') #offset 2.5V
    instrument.write('OUTP1 ON') #ativar output
    instrument.write('SOUR2:FUNC PULS') #igual ao anterior
    instrument.write('SOUR2:BURS:STATe OFF')
    instrument.write('SOUR2:FREQ 2Hz')
    instrument.write('SOUR2:PULS:WIDTH 1000ns')
    instrument.write('OUTP2:POL NORM')
    instrument.write('SOUR2:VOLT:LEV:IMM:AMPL 5VPP')
    instrument.write('SOUR2:VOLT:LEV:IMM:OFFS 2.5V')
    instrument.write('OUTP2 ON') ### initial output of the Function Generator
    keithley.write(':SYST:REM')
    keithley.write(':SOUR:FUNC CURR')  # Set the source function to current
    keithley.write('SOUR:CURR:DEL:AUTO OFF')
    keithley.write(':ROUT:TERM REAR')  # Set the output terminals to the rear panels
    keithley.write(':OUTP ON')  # Turn on the output
    time.sleep(10)
    instrument.write('SOUR1:FREQ 20Hz')
    instrument.write('SOUR2:FREQ 20Hz')
    time.sleep(20)

In [53]:
def startup(A, criticality):
    cont = True #uses the continuation function
    counts = 11 #counts associadas a todas as barras a 0% ###de onde vem este valor??
    k1 = 1 - A/counts # k associado a 11 counts
    initial_values=[counts, (beta[0]*counts)/(Lambda[0]*l), (beta[1]*counts)/(Lambda[1]*l), (beta[2]*counts)/(Lambda[2]*l),
                    (beta[3]*counts)/(Lambda[3]*l), (beta[4]*counts)/(Lambda[4]*l), (beta[5]*counts)/(Lambda[5]*l)] #aproximação de valores iniciais das 7 variáveis
    while 10 < initial_values[0] < 10**12:
        x1 = [k1]*5 ### creating a list with several values of k, to then average them. used due to noise
        k_value = round(root_mean_squared(x1), 5)
        b = time.time()
        while k_value < 1 and cont == True: # subcritical state condition
            #if A/(1-round(root_mean_squared(x1),5))<100 and cont==True:
            z = threadings()[0] #readings from CARAPAU
            del z[-1] #deleting last value from CARAPAU because it's not used

            counts = continuation(initial_values[0], k_value, A, time.time()-b) #next count 
            instrument.write('SOUR1:FREQ' + str(counts + (-10 + random.random()*20)) + 'Hz') #write new frequency baseed on continuation function, noise added for realism
            instrument.write('SOUR1:PULS:DCYC 10') #duty cycle de forma a garantir pulsos de 100ns
            instrument.write('SOUR2:FREQ' + str(counts + (-10 + random.random()*20)) + 'Hz') #write new frequency baseed on continuation function, noise added for realism
            instrument.write('SOUR2:PULS:DCYC 10')
            
            b=time.time()
            initial_values = [counts, (beta[0]*counts)/(Lambda[0]*l),(beta[1]*counts)/(Lambda[1]*l), (beta[2]*counts)/(Lambda[2]*l), (beta[3]*counts)/(Lambda[3]*l), 
                              (beta[4]*counts)/(Lambda[4]*l), (beta[5]*counts)/(Lambda[5]*l)]
            
            print([k_value, initial_values[0], cont]) # !!! OUTPUT: k, counts, bool value of cont  !!!

            k1 = k(z, criticality) #new k 
            del x1[0]
            x1 = x1 + [k1] # now the list of k's includes the new k, will do this for new k's while the cycle iterates
            
                #print([round(root_mean_squared(x1),5),z*20,((1.7/(1-round(root_mean_squared(x1),5))**0.9)*math.exp(-0.00001/(1-round(root_mean_squared(x1),5))))*(0.85+random.random()*0.3),cont])
            
        counts = initial_values[0]
        p = [((k1-1)/k1)]*10 #list of ractivities to average (to decrease noise)
        
        initial_values = [counts, (beta[0]*counts)/(Lambda[0]*l), (beta[1]*counts)/(Lambda[1]*l), (beta[2]*counts)/(Lambda[2]*l), (beta[3]*counts)/(Lambda[3]*l),
                      (beta[4]*counts)/(Lambda[4]*l), (beta[5]*counts)/(Lambda[5]*l)]
        
        return initial_values

In [45]:
b = time.time()

def simulation(A, criticality): #This is a simulation of the bars with the reactor. Variable A is the constant of proportion between the minimal number of counts and k

    instruments()
    startup(A, criticality)

    while 10 < initial_values[0] < 10**12: 

        counts = initial_values[0] 
        previous_values=[counts]*100
        cont = False
        times = []

        for i in range(0, len(previous_values)):
            times = times+[0.09]

            while cont==False and initial_values[0]<10**12: #Second cycle supercritical state.

                w = time.time()
                z = threadings1(sum(p)/len(p), 0.1, initial_values) #CARAPAU readings
                p1 = (cs_1(z[0][0]*20) + cs_2(z[0][1]*20) + cs_3(z[0][2]*20) + cs_4(z[0][3]*20) + cs_r(z[0][4]*20) - criticality) * 10**-5 #new reactivity

                del p[0] 
                p = p + [p1] # now the list of p's includes the new p, will do this for new p's while the cycle iterates
     
                initial_values1 = z[-1]
                del previous_values[0]
                previous_values = previous_values + [initial_values1[0]]
            
                #print(previous_values)
                if abs(previous_values[-1] - previous_values[0]) < 0.1:
                    print([((round(-1/((sum(p)/len(p))-1),5)-1)/(round(-1/((sum(p)/len(p))-1),5)))*10**5,10**8,cont,initial_values1[0]])
                else:
                    print([((round(-1/((sum(p)/len(p))-1), 5)-1)/(round(-1/((sum(p)/len(p))-1),5)))*10**5,cont,sum(times)/(math.log(previous_values[-1]/previous_values[0])), f'{initial_values1[0]:.5e}'])
                initial_values=initial_values1

                k1=round(-1/(p1-1),5)

                if A/(1-k1)>initial_values1[0] and k1<0.9999: #making sure that counts do not go below theoretical level of 1/1-k
                    cont=True                                  #sends back to startup channel
                else:
                    continue
                
                del times[0]
                times=times+[time.time()-w]

In [46]:
r=simulation(A=2, criticality=9093)
instrument.write('OUTP1 OFF')
instrument.write('OUTP2 OFF')

True
True
True
[0.81818, 10.855272303597772, True]
[0.85762, 11.168335784002743, True]
[0.89531, 11.784418370011185, True]
[0.93149, 12.517623272164984, True]
[0.96632, 13.511458352691557, True]
[0.96632, 14.74225028229227, True]
[0.96633, 15.686247411935675, True]
[0.96633, 16.849938059649578, True]
[0.96633, 18.136692651402136, True]
[0.96633, 19.094210685276607, True]
[0.96634, 20.280737837781615, True]
[0.96635, 21.233522408330312, True]
[0.96635, 22.14938867501121, True]
[0.96634, 22.86866651338954, True]
[0.96634, 23.645154571210167, True]
[0.96633, 24.611193473538975, True]
[0.96632, 25.35453885332259, True]
[0.96632, 26.10651445216404, True]
[0.96633, 26.731906828852168, True]
[0.96633, 27.46887495144092, True]
[0.96632, 28.344314765027633, True]
[0.96632, 29.17972222322399, True]
[0.96632, 30.312172524165053, True]
[0.96632, 30.954399796824507, True]
[0.96632, 31.616655990274978, True]
[0.96632, 32.45157730677502, True]
[0.96632, 32.95741725874614, True]
[0.96632, 33.564370770

C:\Users\eduardo.ferreira\AppData\Local\Temp\ipykernel_18580\2281881180.py:87: RuntimeWarning: divide by zero encountered in scalar divide
  if A/(1-k1)>initial_values1[0] and k1<0.9999: #making sure that counts do not go below theoretical level of 1/1-k


[0.0, False, -8983.964971921736, '2.97323e+03']
[0.0, False, -13843.1428079987, '2.97413e+03']
[0.0, False, -31818.69297694043, '2.97507e+03']
[0.0, False, -145073.15388876246, '2.97563e+03']
[0.0, False, 169346.27723153695, '2.97593e+03']
[0.0, False, 53101.00144101364, '2.97623e+03']
[0.0, 100000000, False, 2975.8551522252646]
[0.0, False, 35821.802068871184, '2.97644e+03']
[0.0, False, 24126.179451685035, '2.97675e+03']
[0.0, False, 26568.01654004539, '2.97666e+03']
[0.0, False, 46353.736391187085, '2.97629e+03']
[0.0, False, -76528.27302847615, '2.97549e+03']
[0.0, False, -20422.556641351162, '2.97467e+03']
[-1.00001000009545, False, -12299.741549479912, '2.97392e+03']
[-1.00001000009545, False, -8792.885482487318, '2.97318e+03']
[-1.00001000009545, False, -6818.755594028163, '2.97242e+03']
[-1.00001000009545, False, -9182.884041184614, '2.97329e+03']
[-1.00001000009545, False, -6484.802273955415, '2.97224e+03']
[-1.00001000009545, False, -5308.978967582684, '2.97146e+03']
[-1.0000

KeyboardInterrupt: 